In [1]:
import numpy as np
import importlib

In [2]:
def diffuse_cuts(mc):
    """Applies diffuse dataset cuts on a given monte-carlo data.

    Parameters
    ----------
    mc : str | numpy record ndarray
        Monte-carlo data.

    Returns
    -------
    mc_dc : numpy record ndarray
        Monte-carlo data after diffuse dataset cuts.
    """
    mc_dc = mc[(mc['trueDec'] > np.radians(-5)) &
               (np.log10(mc['trueE']) < 8.0) &
               (mc['sigmaOK'] == 0)]
    return mc_dc

In [3]:
plotter_wkde = importlib.import_module('plotter_wkde_photospline')

#gammas = np.concatenate(([1.5], np.linspace(2.0, 3.0, 11)))
gammas = [1.5, 2.0, 3.0]

logE_list = [2.5, 3.5, 5.0]
sigma_p_list = [0.5, 0.3, 0.2]

base_path = '/home/ge56lag'
output_dir = 'output_Martin'

mc = np.load('/{base_path}/Data/diffuse_northern_tracks_MC_KDE/version-001-p00/dataset_8yr_fit_IC86_2012_16_MC_2017_09_29_more_fields.npy'.format(base_path=base_path))
mc = diffuse_cuts(mc)

for gamma in gammas:
    spatial_KDE_path = '{}/Software/KDE/output/sig_psi_E/adaptive_kd_plotter_wkde_gamma_{}_phi0_1.0/pdf/sig_psi_E.pkl'.format(base_path, gamma)
    norm_KDE_path = '{}/Software/KDE/output/sig_E/adaptive_kd_plotter_wkde_gamma_{}_phi0_1.0/pdf/sig_E.pkl'.format(base_path, gamma)

    spatial_pdf = plotter_wkde.setup(base_path, spatial_KDE_path, norm_KDE_path)

    for logE, sigma_p in zip(logE_list, sigma_p_list):
        try:
            plotter_wkde.make_plot(spatial_pdf, mc, logE, sigma_p, gamma=gamma, delta_sigma=0.2, show_quantile=False, output_dir=output_dir)
        except ZeroDivisionError:
            print("ZeroDivisionError for:")
            print(gamma, logE, sigma_p)

Welcome to JupyROOT 6.14/04


INFO:rootpy.plotting.style] using matplotlib style 'ATLAS'
plotter_wkde_photospline.py:41: RuntimeWarning: divide by zero encountered in divide
  spatial_KDE_vals = spatial_KDE['pdf_vals']/norm_KDE['pdf_vals'][:,np.newaxis,:]
plotter_wkde_photospline.py:41: RuntimeWarning: invalid value encountered in divide
  spatial_KDE_vals = spatial_KDE['pdf_vals']/norm_KDE['pdf_vals'][:,np.newaxis,:]


found 53300 matching events.
2100 events, or 0.0393996247655 % are out of plotting range
114 events, or 0.00213883677298 % are out of histogram bounds
[9.020549444941356, 8.181864456888885, 7.9883974732292415, 7.835866791732763, 7.650794560654021, 7.30745066035068, 6.946748156954918, 6.846673459622244, 6.799712364154883, 6.757047445225239, 6.573546283012585, 6.501607992653313, 6.498983892345539, 6.425883846621104, 6.42214755326904, 6.339725879674065, 6.339468233122235, 6.337126969048476, 6.307889601641824, 6.270355823562833, 6.230443380061416, 6.201092791610531, 6.125697142072366, 6.119396999798139, 6.074209824179701, 6.063802676390134, 6.0235065386032325, 6.017298040694182, 5.9527093888657285, 5.936196182756615, 5.935194458121281, 5.876277323469649, 5.860309948693413, 5.817872884524366, 5.808688605842091, 5.803099485052518, 5.798728989981373, 5.787809865761284, 5.761602688454621, 5.700983602663112, 5.6948138770238526, 5.686573703797984, 5.662773921722252, 5.64200616470263, 5.640190852

/home/ge56lag/py2/lib/python2.7/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)
/home/ge56lag/py2/lib/python2.7/site-packages/matplotlib/font_manager.py:1331: UserWarning: findfont: Font family [u'sans-serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


found 22832 matching events.
870 events, or 0.0381044148563 % are out of plotting range
165 events, or 0.00722669936931 % are out of histogram bounds
[12.41051144890517, 7.906529479385936, 7.606588886905591, 7.462933518429186, 6.59116314540087, 6.536388826592491, 6.456625223320444, 6.366129575696905, 5.930486605861951, 5.804863320130014, 5.796269799230805, 5.773532192343948, 5.702641708053613, 5.599477103069061, 5.553318642295504, 5.4856695182574065, 5.234536466673288, 5.176637283765948, 5.04489277388596, 5.008102280586733, 4.990096269348767, 4.942550489837139, 4.939736099410673, 4.862110934147003, 4.8439868808676145, 4.79473537800332, 4.790903238713635, 4.778323391607514, 4.734732297895247, 4.700293429543393, 4.6821447487341326, 4.668659471086367, 4.650023766042489, 4.618681710283869, 4.575840404983848, 4.554281412474415, 4.542938187235277, 4.500302638667894, 4.463999505197313, 4.437339807605613, 4.432436110784498, 4.403572313731035, 4.400009044469182, 4.390623308242082, 4.37913960409

found 4395 matching events.
72 events, or 0.0163822525597 % are out of plotting range
26 events, or 0.00591581342435 % are out of histogram bounds
[7.385369516401584, 5.659794570203385, 4.579782377346181, 4.534870645281853, 4.3740995809157095, 3.9522017326966568, 3.8942910974896874, 3.8498974700042443, 3.734534285544038, 3.7172403012376063, 3.630034583689876, 3.5987700103338924, 3.44914319719314, 3.0108698682974, 3.0023126334725307, 2.9258191628293013, 2.724311279910718, 2.6693985585921745, 2.6623144651475386, 2.615604451487949, 2.6137050684534624, 2.4413662418416813, 2.3560244026844597, 2.126515272348191, 2.046524067204123, 2.0237863278962998]
found 53300 matching events.
2100 events, or 0.0393996247655 % are out of plotting range
114 events, or 0.00213883677298 % are out of histogram bounds
[9.020549444941356, 8.181864456888885, 7.9883974732292415, 7.835866791732763, 7.650794560654021, 7.30745066035068, 6.946748156954918, 6.846673459622244, 6.799712364154883, 6.757047445225239, 6.573

### Hans part

In [ ]:
plotter_wkde = importlib.import_module('plotter_wkde_photospline')

gammas = [1.5, 2.0, 2.5, 3.0]

xpts = np.array([2.5 + i*0.5 for i in range(9)])
ypts = np.array([1.4, 1.1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.4])
tymin = 0.1

base_path = '/home/ge56lag'
output_dir = 'output_Hans/diffuse_cut/photospline'

mc = np.load('/{base_path}/Data/diffuse_northern_tracks_MC_KDE/version-001-p00/dataset_8yr_fit_IC86_2012_16_MC_2017_09_29_more_fields.npy'.format(base_path=base_path))

mc = diffuse_cuts(mc)

for gamma in gammas:
    spatial_KDE_path = '{}/Software/KDE/output/sig_psi_E/adaptive_kd_plotter_wkde_gamma_{}_phi0_1.0/pdf/sig_psi_E.pkl'.format(base_path, gamma)
    norm_KDE_path = '{}/Software/KDE/output/sig_E/adaptive_kd_plotter_wkde_gamma_{}_phi0_1.0/pdf/sig_E.pkl'.format(base_path, gamma)

    spatial_pdf = plotter_wkde.setup(base_path, spatial_KDE_path, norm_KDE_path)

    for obj in zip(xpts, ypts):
        logE, tymax = obj
        for sigma_p in np.linspace(tymin, tymax, 10):
            try:
                plotter_wkde.make_plot(spatial_pdf, mc, logE, sigma_p, gamma=gamma, delta_sigma=0.2, show_quantile=False, output_dir=output_dir)
            except ZeroDivisionError:
                print("ZeroDivisionError for:")
                print(gamma, logE, sigma_p)

### Sigma_p given E_rec

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
base_path = '/home/ge56lag'

mc = np.load('/{base_path}/Data/diffuse_northern_tracks_MC_KDE/version-001-p00/dataset_8yr_fit_IC86_2012_16_MC_2017_09_29_more_fields.npy'.format(base_path=base_path))
mc = diffuse_cuts(mc)

In [ ]:
fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(111)
h = ax.hist2d(mc['logE'], mc['sigma_pull_corrected']/np.pi*180.0, norm=mpl.colors.LogNorm(), bins=[100, 100], range=[[2, 6], [0, 2]])

fig.colorbar(h[3], ax=ax)